# UIT DSC

## Preprocessing

In [1]:
import pandas as pd
import numpy as np
import json

In [2]:
# Load the original file
input_file_path = '/APTTH36781/vimmsd-train.json'

In [3]:
# Initialize empty dictionaries for the new files
text_sarcasm = {}
image_sarcasm = {}
multi_sarcasm = {}
not_sarcasm = {}

# Open and read the original file
with open(input_file_path, 'r', encoding='utf-8') as file:
    data = json.load(file)
    
    # Iterate through the dataset and distribute based on labels
    for key, value in data.items():
        if value['label'] == 'text-sarcasm':
            text_sarcasm[key] = value
        elif value['label'] == 'image-sarcasm':
            image_sarcasm[key] = value
        elif value['label'] == 'multi-sarcasm':
            multi_sarcasm[key] = value
        elif value['label'] == 'not-sarcasm':
            not_sarcasm[key] = value

# Save each label-specific data to a separate file
output_file_paths = {
    'text_sarcasm': '/APTTH36781/text_sarcasm.json',
    'image_sarcasm': '/APTTH36781/image_sarcasm.json',
    'multi_sarcasm': '/APTTH36781/multi_sarcasm.json',
    'not_sarcasm': '/APTTH36781/not_sarcasm.json'
}

with open(output_file_paths['text_sarcasm'], 'w', encoding='utf-8') as file:
    json.dump(text_sarcasm, file, ensure_ascii=False, indent=4)
    
with open(output_file_paths['image_sarcasm'], 'w', encoding='utf-8') as file:
    json.dump(image_sarcasm, file, ensure_ascii=False, indent=4)
    
with open(output_file_paths['multi_sarcasm'], 'w', encoding='utf-8') as file:
    json.dump(multi_sarcasm, file, ensure_ascii=False, indent=4)
    
with open(output_file_paths['not_sarcasm'], 'w', encoding='utf-8') as file:
    json.dump(not_sarcasm, file, ensure_ascii=False, indent=4)

output_file_paths

{'text_sarcasm': '/APTTH36781/text_sarcasm.json',
 'image_sarcasm': '/APTTH36781/image_sarcasm.json',
 'multi_sarcasm': '/APTTH36781/multi_sarcasm.json',
 'not_sarcasm': '/APTTH36781/not_sarcasm.json'}

## Model

### Pretrained Model

OCR

In [3]:
import json
import easyocr
import os

In [19]:
# Đường dẫn thư mục gốc chứa các ảnh
image_directory = "D:/APTTH36781/public_test/dev-images/"

# Hàm trích xuất văn bản từ ảnh
def text_in_image(image_path):
    reader = easyocr.Reader(['vi'], gpu=True)  # Khởi tạo EasyOCR với tiếng Việt
    result = reader.readtext(image_path, detail=0)  # Trích xuất văn bản (detail=0 để chỉ lấy văn bản)
    return result

# Mở file JSON
with open('vimmsd-public-test.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

# Duyệt qua từng mục trong file JSON
for key, value in data.items():
    image_name = value.get("image")  # Lấy tên file ảnh từ mục "image"
    
    if image_name:
        image_path = os.path.join(image_directory, image_name)  # Kết hợp đường dẫn gốc với tên file
        
        if os.path.exists(image_path):  # Kiểm tra nếu ảnh tồn tại
            extracted_texts = text_in_image(image_path)  # Trích xuất văn bản từ ảnh
            text_in_image_str = " ".join(extracted_texts)  # Nối danh sách thành chuỗi ký tự
            
            # Thêm chuỗi văn bản trích xuất được vào mục "text_in_image"
            data[key]["text_in_image"] = text_in_image_str
        else:
            print(f"File not found: {image_path}")

# Ghi lại file JSON với text_in_image đã được thêm vào
with open('output_file.json', 'w', encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=4)

print("Dữ liệu đã được lưu thành công vào file JSON.")

Dữ liệu đã được lưu thành công vào file JSON.


In [27]:
# Đường dẫn thư mục gốc chứa các ảnh
image_directory = "D:/APTTH36781/train-images/"

# Hàm trích xuất văn bản từ ảnh
def text_in_image(image_path):
    reader = easyocr.Reader(['vi'], gpu=True)  # Khởi tạo EasyOCR với tiếng Việt
    result = reader.readtext(image_path, detail=0)  # Trích xuất văn bản (detail=0 để chỉ lấy văn bản)
    return result

# Mở file JSON
with open('vimmsd-train.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

# Duyệt qua từng mục trong file JSON
for key, value in data.items():
    image_name = value.get("image")  # Lấy tên file ảnh từ mục "image"
    
    if image_name:
        image_path = os.path.join(image_directory, image_name)  # Kết hợp đường dẫn gốc với tên file
        
        if os.path.exists(image_path):  # Kiểm tra nếu ảnh tồn tại
            extracted_texts = text_in_image(image_path)  # Trích xuất văn bản từ ảnh
            text_in_image_str = " ".join(extracted_texts)  # Nối danh sách thành chuỗi ký tự
            
            # Thêm chuỗi văn bản trích xuất được vào mục "text_in_image"
            data[key]["text_in_image"] = text_in_image_str
        else:
            print(f"File not found: {image_path}")

# Ghi lại file JSON với text_in_image đã được thêm vào
with open('output_file_train.json', 'w', encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=4)

print("Dữ liệu đã được lưu thành công vào file JSON.")

Dữ liệu đã được lưu thành công vào file JSON.


PhoBert + ResNet

In [1]:
import torch

# Kiểm tra xem GPU có sẵn không và thiết lập thiết bị
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Sử dụng thiết bị: {device}")

Sử dụng thiết bị: cuda


In [9]:
from transformers import AutoModel, AutoTokenizer
from torchvision import models, transforms
from PIL import Image
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import json
import os
from tqdm import tqdm

# Load PhoBERT-base for text
phobert_tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")
phobert_model = AutoModel.from_pretrained("vinai/phobert-base").to(device)

# Load a pre-trained ResNet for image processing
resnet_model = models.resnet50(pretrained=True).to(device)
resnet_model.fc = nn.Identity()  # Remove the classification layer to get features

# Define transformation for image preprocessing
image_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Custom dataset class
class SarcasmDataset(Dataset):
    def __init__(self, json_data, image_folder, transform=None):
        # Chuyển đổi json_data thành danh sách nếu nó không phải là danh sách
        if isinstance(json_data, dict):
            self.data = list(json_data.values())  # Hoặc danh sách các giá trị
        else:
            self.data = json_data
            
        self.image_folder = image_folder
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        if idx < 0 or idx >= len(self.data):
            raise IndexError("Index out of bounds")

        item = self.data[idx]

        if "image" not in item or "label" not in item or "caption" not in item:
            raise KeyError(f"Missing required fields in item at index {idx}")

        image_path = os.path.join(self.image_folder, item["image"])

        try:
            image = Image.open(image_path).convert("RGB")
            if self.transform:
                image = self.transform(image)
        except Exception as e:
            print(f"Error loading image {image_path}: {e}")
            image = torch.zeros(3, 224, 224)

        # Ánh xạ các chuỗi nhãn thành số nguyên
        label_mapping = {
            "multi-sarcasm": 0,
            "not-sarcasm": 1,
            "image-sarcasm": 2,
            "text-sarcasm": 3
        }


        label_str = item["label"]
        if label_str not in label_mapping:
            raise ValueError(f"Invalid label '{label_str}' at index {idx}")
        label = torch.tensor(label_mapping[label_str])

        caption = item["caption"]
        text_in_image = item.get("text_in_image", "")

        return image, caption, text_in_image, label


# Custom classification head for combining text and image features
class SarcasmClassifier(nn.Module):
    def __init__(self, text_feature_size, image_feature_size, num_classes=4):
        super(SarcasmClassifier, self).__init__()
        self.fc = nn.Linear(text_feature_size + image_feature_size, num_classes)

    def forward(self, text_features, image_features):
        combined_features = torch.cat((text_features, image_features), dim=1)
        return self.fc(combined_features)

# Load the training data
train_json_path = r"D:/APTTH36781/output_file_train.json"
train_image_folder = r"D:/APTTH36781/train-images"

with open(train_json_path, 'r', encoding='utf-8') as f:
    train_data = json.load(f)

# Create dataset and dataloader
train_dataset = SarcasmDataset(train_data, train_image_folder, transform=image_transforms)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Create the classifier model
classifier = SarcasmClassifier(text_feature_size=768, image_feature_size=2048, num_classes=4).to(device)

# Define optimizer and loss function
optimizer = optim.Adam(classifier.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

# Training loop
num_epochs = 10  # Adjust as needed
classifier.train()  # Set model to training mode
for epoch in range(num_epochs):
    running_loss = 0.0
    correct_predictions = 0
    total_samples = 0

    for images, captions, text_in_images, labels in tqdm(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        # Get features for images
        with torch.no_grad():
            image_features = resnet_model(images)

        # Process the captions
        inputs = phobert_tokenizer(captions, return_tensors="pt", padding=True, truncation=True, max_length=128)
        inputs = {key: value.to(device) for key, value in inputs.items()}
        with torch.no_grad():
            text_features = phobert_model(**inputs).pooler_output.to(device)

        # Forward pass
        optimizer.zero_grad()
        logits = classifier(text_features, image_features)

        # Compute loss
        loss = criterion(logits, labels)
        running_loss += loss.item()

        # Compute accuracy
        _, predicted = torch.max(logits, 1)
        correct_predictions += (predicted == labels).sum().item()
        total_samples += labels.size(0)

        # Backward pass
        loss.backward()
        optimizer.step()

    avg_loss = running_loss / len(train_loader)
    accuracy = correct_predictions / total_samples
    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {avg_loss:.4f}, Accuracy: {accuracy:.4f}")

# Save the trained model
torch.save(classifier.state_dict(), 'sarcasm_classifier.pth')
print("Model trained and saved.")

  0%|          | 0/338 [00:00<?, ?it/s]d:\Python312\Lib\site-packages\transformers\models\roberta\modeling_roberta.py:370: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
100%|██████████| 338/338 [05:05<00:00,  1.11it/s]


Epoch 1/10, Loss: 0.7963, Accuracy: 0.6311


100%|██████████| 338/338 [04:41<00:00,  1.20it/s]


Epoch 2/10, Loss: 0.7373, Accuracy: 0.6681


100%|██████████| 338/338 [04:45<00:00,  1.18it/s]


...

Epoch 8/10, Loss: 0.6695, Accuracy: 0.6950


100%|██████████| 338/338 [04:38<00:00,  1.21it/s]


Epoch 9/10, Loss: 0.6661, Accuracy: 0.6913


100%|██████████| 338/338 [04:37<00:00,  1.22it/s]

Epoch 10/10, Loss: 0.6636, Accuracy: 0.6988
Model trained and saved.


In [10]:
# Tải mô hình đã huấn luyện
classifier.load_state_dict(torch.load('sarcasm_classifier.pth'))
classifier.eval()  # Đặt mô hình vào chế độ dự đoán

# Prediction function
def predict_sarcasm(image_path, caption, text_in_image=None):
    # Process the image
    try:
        image = Image.open(image_path).convert("RGB")
    except FileNotFoundError:
        print(f"Image not found: {image_path}")
        return None

    image_tensor = image_transforms(image).unsqueeze(0).to(device)

    # Extract image features using ResNet
    with torch.no_grad():
        image_features = resnet_model(image_tensor).to(device)

    # Process the caption using PhoBERT
    inputs = phobert_tokenizer(caption, return_tensors="pt", padding=True, truncation=True, max_length=128)
    inputs = {key: value.to(device) for key, value in inputs.items()}  # Chuyển các tensor văn bản sang thiết bị
    with torch.no_grad():
        text_features = phobert_model(**inputs).pooler_output.to(device)

    # Check if text_in_image exists and process it similarly to caption
    text_features_image_text = None
    if text_in_image:
        inputs_image_text = phobert_tokenizer(text_in_image, return_tensors="pt", padding=True, truncation=True, max_length=128)
        inputs_image_text = {key: value.to(device) for key, value in inputs_image_text.items()}
        with torch.no_grad():
            text_features_image_text = phobert_model(**inputs_image_text).pooler_output.to(device)

    # Predict using the classifier
    with torch.no_grad():
        logits = classifier(text_features, image_features)
        predicted_label = torch.argmax(logits, dim=1).item()

        # If there is text_in_image, also predict sarcasm for it
        if text_features_image_text is not None:
            logits_image_text = classifier(text_features_image_text, image_features)
            predicted_label_image_text = torch.argmax(logits_image_text, dim=1).item()

            # If both caption and text_in_image are sarcastic, return "multi-sarcasm"
            if predicted_label_image_text == 3 and predicted_label == 2:
                return 0  # "multi-sarcasm"

    return predicted_label

# Function to read JSON file, predict, and save results
def process_json(json_path, image_folder, output_json, phase="dev"):
    # Load JSON data
    with open(json_path, 'r', encoding='utf-8') as f:
        data = json.load(f)

    label_mapping = {
        0: "multi-sarcasm",
        1: "not-sarcasm",
        2: "image-sarcasm",
        3: "text-sarcasm"
    }

    # Iterate through each item in the JSON
    results = {}
    for id, item in data.items():
        image_name = item["image"]
        caption = item["caption"]
        label = item.get("label")  # Get label if available, handle null cases
        text_in_image = item.get("text_in_image", "")

        # Create full image path
        image_path = os.path.join(image_folder, image_name)

        # Predict label only if label is null
        if label is None:
            predicted_label = predict_sarcasm(image_path, caption, text_in_image)
            results[id] = label_mapping[predicted_label] if predicted_label is not None else "Image not found"
        else:
            results[id] = label  # Use the existing label if available

    # Final output data in the specified format
    output_data = {
        "results": results,
        "phase": phase
    }

    # Save results to output JSON
    with open(output_json, 'w', encoding='utf-8') as f:
        json.dump(output_data, f, ensure_ascii=False, indent=4)

    print(f"Results saved to {output_json}")

# Example usage
json_input_path = "D:\\APTTH36781\\output_file.json"
image_folder_path = "D:\\APTTH36781\\public_test\\dev-images"
output_json_path = "output_predictions_test.json"

process_json(json_input_path, image_folder_path, output_json_path)

C:\Users\QuynhAnh\AppData\Local\Temp\ipykernel_20748\3969397492.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  classifier.load_state_dict(torch.load('sarcasm_classifier

Results saved to output_predictions_test.json
